In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_fixed
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName):
    path = "old_results/stream/fixed/gridsearch_amanda_fixed_EVL-{}.txt".format(clfName)
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    #datasets=[setup.loadKeystroke]
    #batches=8
    #datasets=[setup.loadNOAADataset]
    #batches=50
    
    arrClfName = ['LP', 'KNN']#['SGD', 'NB', 'RF', 'LP', 'KNN']
    for clfName in arrClfName:
        print("**************** BEGIN of {} results ****************".format(clfName))
        poolSize = 300
        isBatchMode = False
        #testing grid search
        for i in range(len(datasets)):
            initialLabeledData = 50
            if i==len(datasets)-3 or i==len(datasets)-2 or i==len(datasets)-1:
                initialLabeledData=150
                
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)
            
            sizeOfBatch = int((len(dataLabels)-initialLabeledData)/batches)

            print("{}: pool with {} instances of {}".format(description, poolSize, sizeOfBatch))

            tuned_params = [{"excludingPercentage" : [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5],
                             "sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize],
                             "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData], "clfName":[clfName]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_fixed.run(**g)

                try:
                    gs.fit(dataValues, dataLabels)
                    averageAccuracy = np.mean(gs.predict())
                    #print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName)
        print("******** END of {} results ********".format(clfName))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP results ****************
One Class Diagonal Translation. 2 Dimensional data.: 100 batches of 159 instances


C:\Users\raul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


99.8558
{'K': 4, 'batches': 100, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
One Class Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.2855
{'K': 5, 'batches': 100, 'clfName': 'LP', 'excludingPercentage': 0.55, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Two Classes Diagonal Translation. 2 Dimensional data: 100 batches of 159 instances
92.821
{'K': 7, 'batches': 100, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Two Classes Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
78.6156
{'K': 2, 'batches': 100, 'clfName': 'LP', 'excludingPercentage': 0.9, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Four Classes Rotating Separated. Bidimensional.: 100 batches of 1443 instances
99.9909
{'K': 3,

C:\Users\raul\Desktop\GITHUB\Systems-Engineering\Dissertation\source\classifiers.py:135: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return KNeighborsClassifier(n_neighbors=K, algorithm = 'brute').fit(X, y)


99.8432
{'K': 9, 'batches': 100, 'clfName': 'KNN', 'excludingPercentage': 0.85, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
One Class Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.2916
{'K': 9, 'batches': 100, 'clfName': 'KNN', 'excludingPercentage': 0.85, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Two Classes Diagonal Translation. 2 Dimensional data: 100 batches of 159 instances
90.2623
{'K': 9, 'batches': 100, 'clfName': 'KNN', 'excludingPercentage': 0.9, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Two Classes Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
77.243
{'K': 12, 'batches': 100, 'clfName': 'KNN', 'excludingPercentage': 0.9, 'initialLabeledData': 50, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 159}
Four Classes Rotating Separated. Bidimensional.: 100 batches of 1443 instances
99.9916
{'